In [2]:
# We use Numpy for  it's wide range of built-in mathematical operations, and array-structure
import numpy as np
# We use Sympy for symbolic manipulation - in a similar manner to Mathematica
import sympy as sym
import sympy.core
from sympy.solvers.solveset import linsolve
# We tell sympy that when printing here, try to ormat as LaTeX
sym.init_printing(use_latex=True)

## We define a function 'analytic_J', which simply returns the minimised $J$
## value obtained from the two analytical solutions

In [3]:
def analytic_J(): # Need to update
    alpha = (np.exp(2) - 2)/(np.exp(2) - np.exp(-2))
    a = (1-alpha)**2
    b = np.exp(4)-1
    c = alpha**2
    d = 1 - np.exp(-4)
    return a*b + c*d
analytic_J()

## We define a function 'analytic_y', which takes an argument 'x', and returns  
## $y(x)$, where $y$ is the minimising function obtained in the analytical solutions.
## This is useful for comparing and plotting

In [4]:
def analytic_y(x):
    alpha = (np.exp(2) - 2)/(np.exp(2) - np.exp(-2))
    return ((1 - alpha)*np.exp(x) + alpha*np.exp(-1*x))

assert (np.isclose(analytic_y(0),1.0) and np.isclose(analytic_y(2),2.0)), "y does not match boundary conditions"  
# Alert if y doesn't match the boundary conditions

## We define 'defn_F' to take arguments of $y$ and $\frac{dy}{dx}$, and return the integrand for
## these values. This is used in both the symbolic and numerical calculations of $J$

In [5]:
def defn_F(y, y_dash):
    return (y_dash**2 + y**2)

# Initial Working:
Here is the 'testing ground' for getting this whole thing to work. 

First I define some constants used in our problem, like boundary conditions, intervals, etc. - just so as I can easily change them when trying to expand to more general cases.

Then I define two functions, 'get_J' and 'get_J_deriv_symbolic', which return the symbolic/semi-symbolic/numeric approximation of $J$, and the derivative of a symbolic/semi-symbolic $J$ approximation with respect to a given symbol. (respectively)

I then define 2equivalent sets of symbolic y-points/variables; ys and ys_permanent_symbols - the idea is I will swap ys values for numeric values as they are discovered, but treat ys_permanent_symbolic as immutable so as to always have a reference t oany given symbol.

I work out an initial $J$ estimate using ys; so at this point the $J$ estimate is entirely symbolic.
A list, 'J_derivs', of $\frac{dJ}{dy_{i}}$ is built up.

These J_derivs are equated to $0$ and solved for the symbol they were differentiated w.r.t., and these new $y$ values are stored in ys_optimised_symbolic.

The $y(2)=2$ boundary condition is applied to the ys list, and this is then used to update the values of the ys_optimised_symbolic list through back propogation.

The $y(0)=1$ boundary condition is then applied and to the ys list, and this is then used to update ys_optimised_symbolic through forward propogation.

At this point, each value in ys_optimised_symbolic is optimised symbolically, and has had it's values updated to reflect the impact of applying the boundary conditions. We now solve this list of symbolic expressions as a system of linear equations; we could do this manually, implementing Gaussian elimination or some equivalent method (QR maybe), but why bother when Sympy has an inbuilt method - linsolve. So we simply pass in our list to this, and get back the solved values! 

Now that we have these as numeric values, we can simply pass them into get_J to get a numeric solution for the minimised $J$ value.

In [6]:
"""
Notes:
>uses Sympy, for symbolic manipulation, 
usage very similar to Mathematica.
>Further reading available here:
http://docs.sympy.org/latest/index.html
"""
num_x_pts = 3
y_start,y_end = 1,2 # limits of integral
a,b = 0,2
x_pts = np.linspace(a,b,num_x_pts); # create 3 evenly-spaced points to be our x-coordinates
h = x_pts[1]-x_pts[0]

def get_J(ys, h, defn_F): # Uses 2-step finite difference method to estimate J. Works numerically & symbolically
    J = 0
    for i in range(len(ys)-1):
        J += defn_F(0.5*(ys[i]+ys[i+1]),(ys[i+1]-ys[i])/h)*h
    return sym.simplify(J)

def get_J_deriv_symbolic(J_symbolic, symbol): # Returns the derivative of a symbolic J estimate, w.r.t. 'symbol'
    return J_symbolic.diff(symbol)

ys = list(sym.symbols('y:{}'.format(num_x_pts)))
ys_permanent_symbols = sym.symbols('y:{}'.format(num_x_pts))
J = get_J(ys, h, defn_F)

J_derivs = [get_J_deriv_symbolic(J, y) for y in ys]
ys_optimised_symbolic = [sym.solve(J_d, ys[i])[0] for i,J_d in enumerate(J_derivs)]

#2nd boundary condition & back propogation:
ys[-1] = 2

for i in list(range(len(ys))[::-1]):
    ys_optimised_symbolic = [y.subs(str(ys_permanent_symbols[i]),ys[i]) for y in ys_optimised_symbolic]

#1st boundary & forward propagation:
ys[0] = 1
for i in range(len(ys)):
    ys_optimised_symbolic = [y.subs(str(ys_permanent_symbols[i]),ys[i]) if not isinstance(ys[i],sym.mul.Mul) else y for y in ys_optimised_symbolic]

for i,y in enumerate(ys_optimised_symbolic):
    if not isinstance(y,sym.mul.Mul):
        ys[i] = y

ysols = [ys[i] - ys_permanent_symbols[i] for i in range(len(ys))]
sols = linsolve(ysols,ys_permanent_symbols)#[0],ys_permanent_symbols[-1]])
#J_derivs = [get_J_deriv_symbolic(J, y) for y in ys_permanent_symbols[1:-1]]
#ys_optimised_symbolic = [sym.solve(J_d, ys_permanent_symbols[i])[0] for i,J_d in enumerate(J_derivs)]
#for i in range(len(ys)):
#    ys_optimised_symbolic = [y.subs(str(ys_permanent_symbols[i]),ys[i]) if not isinstance(ys[i],sym.mul.Mul) else y for y in ys_optimised_symbolic]
#ys_optimised_symbolic
sols = [s for s in sols]
sols = sols[0]
print([sol for sol in sols])
J_update = get_J(sols, h, defn_F)
J_update

[1, 0.900000000000000, 2]


## Attempt at abstracting the above craic into a function, as a first step towards abstracting it to a more general case:

In [7]:
def magically_solve(F,n,interval_end_pts_tuple,y_end_pts_tuple):
    num_x_pts = n
    y_start,y_end = y_end_pts_tuple
    a,b = interval_end_pts_tuple
    x_pts = np.linspace(a,b,num_x_pts); 
    h = x_pts[1]-x_pts[0]
    
    ys = list(sym.symbols('y:{}'.format(num_x_pts)))
    ys_permanent_symbols = sym.symbols('y:{}'.format(num_x_pts))
    J = get_J(ys, h, F)
    J_derivs = [get_J_deriv_symbolic(J, y) for y in ys]
    ys_optimised_symbolic = [sym.solve(J_d, ys[i])[0] for i,J_d in enumerate(J_derivs)]
    
    ys[-1] = y_end
    for i in list(range(len(ys))[::-1]):
        ys_optimised_symbolic = [y.subs(str(ys_permanent_symbols[i]),ys[i]) for y in ys_optimised_symbolic]
    ys[0] = y_start
    for i in range(len(ys)):
        ys_optimised_symbolic = [y.subs(str(ys_permanent_symbols[i]),ys[i]) if not isinstance(ys[i],(sym.mul.Mul,sym.add.Add)) else y for y in ys_optimised_symbolic]
    for i,y in enumerate(ys_optimised_symbolic):
        if not isinstance(y,(sym.mul.Mul)):
            ys[i] = y
    ysols = [ys[i] - ys_permanent_symbols[i] for i in range(len(ys))]
    sols = [s for s in linsolve(ysols,ys_permanent_symbols)][0]
    return get_J(sols, h, F)


In [8]:
magically_solve(defn_F,5,(0,2),(1,2))

## We now define a couple of functions to evaluate the accuracy of approximations:

In [72]:
def get_perc_diff(estimate):
    real = analytic_J()
    abs_diff = real - estimate
    perc_diff = 100*abs_diff/real
    return np.abs(perc_diff)

In [11]:
get_perc_diff(J_update)

In [12]:
def get_abs_diff(estimate):
    real = analytic_J()
    abs_diff = real - estimate
    return abs_diff

In [13]:
get_abs_diff(J_update)